In [1]:
import sys
import os
import onnx
from maraboupy import Marabou
from maraboupy import MarabouNetworkONNX
import tensorflow as tf
import numpy as np
import tf2onnx
from tqdm import tqdm

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

In [3]:
network = Marabou.read_onnx("fashion_mnist_classification_model_tf.onnx")

In [4]:
y_test[0]

9

In [5]:
input_9 = x_test[0].flatten()
input_9

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

Iterative example for all class

In [6]:
for i, input_vals in tqdm(enumerate(x_test)):
    correct_class = y_test[i]
    input_vals = input_vals.flatten()
    print(f'{i}/{len(x_test)}')
    print('class:', correct_class)
    print('result:')
    
    # Load the ONNX model
    network = Marabou.read_onnx("fashion_mnist_classification_model_tf.onnx")

    # Apply input bounds
    input_vars = network.inputVars[0].flatten()
    beta = 0.5  # Set the upper bound by beta
    for i, (input_var, input_val) in enumerate(zip(input_vars, input_vals)):
        if input_val == 0.0:
            network.setLowerBound(input_var, -beta)
            network.setUpperBound(input_var, beta)
        else:
            network.setLowerBound(input_var, input_val - beta)
            network.setUpperBound(input_var, input_val + beta)

    output_vars = network.outputVars[0][0]
    for i in range(0, 10, 1):
        other_class = i  # Compare y9 with y0
        network.addInequality(
            [output_vars[other_class], output_vars[correct_class]],  # [y_0, y_9]
            [1, -1],
            0
        )

        result, values, statistics = network.solve(verbose=False)
        # print(statistics)

        # Interpret results
        if result == "SAT":
            print("SAT: Solution met the constraint. Counterexample founds!")
            print("Variable values satisfying the constraints:")
            break
        elif result == "UNSAT":
            print("UNSAT: No solution satisfies the constraints. The current network is robust for the input range")
            break
        else:
            print("Could not determine satisfiability. Continue on the next instance!")
        print()
        


0it [00:00, ?it/s]

0/10000
class: 9
result:
Engine::processInputQuery: Input query (before preprocessing): 139 equations, 1060 variables
Engine::processInputQuery: Input query (after preprocessing): 267 equations, 1189 variables

Input bounds:
	x0: [ -0.5000,   0.5000] 
	x1: [ -0.5000,   0.5000] 
	x2: [ -0.5000,   0.5000] 
	x3: [ -0.5000,   0.5000] 
	x4: [ -0.5000,   0.5000] 
	x5: [ -0.5000,   0.5000] 
	x6: [ -0.5000,   0.5000] 
	x7: [ -0.5000,   0.5000] 
	x8: [ -0.5000,   0.5000] 
	x9: [ -0.5000,   0.5000] 
	x10: [ -0.5000,   0.5000] 
	x11: [ -0.5000,   0.5000] 
	x12: [ -0.5000,   0.5000] 
	x13: [ -0.5000,   0.5000] 
	x14: [ -0.5000,   0.5000] 
	x15: [ -0.5000,   0.5000] 
	x16: [ -0.5000,   0.5000] 
	x17: [ -0.5000,   0.5000] 
	x18: [ -0.5000,   0.5000] 
	x19: [ -0.5000,   0.5000] 
	x20: [ -0.5000,   0.5000] 
	x21: [ -0.5000,   0.5000] 
	x22: [ -0.5000,   0.5000] 
	x23: [ -0.5000,   0.5000] 
	x24: [ -0.5000,   0.5000] 
	x25: [ -0.5000,   0.5000] 
	x26: [ -0.5000,   0.5000] 
	x27: [ -0.5000,   0.5000] 
	

1it [00:38, 38.80s/it]

Could not determine satisfiability. Continue on the next instance!

Engine::processInputQuery: Input query (after preprocessing): 275 equations, 1197 variables

Input bounds:
	x0: [ -0.5000,   0.5000] 
	x1: [ -0.5000,   0.5000] 
	x2: [ -0.5000,   0.5000] 
	x3: [ -0.5000,   0.5000] 
	x4: [ -0.5000,   0.5000] 
	x5: [ -0.5000,   0.5000] 
	x6: [ -0.5000,   0.5000] 
	x7: [ -0.5000,   0.5000] 
	x8: [ -0.5000,   0.5000] 
	x9: [ -0.5000,   0.5000] 
	x10: [ -0.5000,   0.5000] 
	x11: [ -0.5000,   0.5000] 
	x12: [ -0.5000,   0.5000] 
	x13: [ -0.5000,   0.5000] 
	x14: [ -0.5000,   0.5000] 
	x15: [ -0.5000,   0.5000] 
	x16: [ -0.5000,   0.5000] 
	x17: [ -0.5000,   0.5000] 
	x18: [ -0.5000,   0.5000] 
	x19: [ -0.5000,   0.5000] 
	x20: [ -0.5000,   0.5000] 
	x21: [ -0.5000,   0.5000] 
	x22: [ -0.5000,   0.5000] 
	x23: [ -0.5000,   0.5000] 
	x24: [ -0.5000,   0.5000] 
	x25: [ -0.5000,   0.5000] 
	x26: [ -0.5000,   0.5000] 
	x27: [ -0.5000,   0.5000] 
	x28: [ -0.5000,   0.5000] 
	x29: [ -0.5000,   0.50

1it [01:02, 62.04s/it]

Engine::processInputQuery: Input query (before preprocessing): 145 equations, 1060 variables
Engine::processInputQuery: Input query (after preprocessing): 272 equations, 1194 variables

Input bounds:
	x0: [ -0.5000,   0.5000] 
	x1: [ -0.5000,   0.5000] 
	x2: [ -0.5000,   0.5000] 
	x3: [ -0.5000,   0.5000] 
	x4: [ -0.5000,   0.5000] 
	x5: [ -0.5000,   0.5000] 
	x6: [ -0.5000,   0.5000] 
	x7: [ -0.5000,   0.5000] 
	x8: [ -0.5000,   0.5000] 
	x9: [ -0.5000,   0.5000] 
	x10: [ -0.4490,   0.5510] 
	x11: [ -0.2373,   0.7627] 
	x12: [ -0.5000,   0.5000] 
	x13: [ -0.5000,   0.5000] 
	x14: [ -0.5000,   0.5000] 
	x15: [ -0.5000,   0.5000] 
	x16: [ -0.3039,   0.6961] 
	x17: [ -0.3510,   0.6490] 
	x18: [ -0.5000,   0.5000] 
	x19: [ -0.5000,   0.5000] 
	x20: [ -0.5000,   0.5000] 
	x21: [ -0.5000,   0.5000] 
	x22: [ -0.5000,   0.5000] 
	x23: [ -0.5000,   0.5000] 
	x24: [ -0.5000,   0.5000] 
	x25: [ -0.5000,   0.5000] 
	x26: [ -0.5000,   0.5000] 
	x27: [ -0.5000,   0.5000] 
	x28: [ -0.5000,   0.5000]

KeyboardInterrupt: 